## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier

import random
import warnings

warnings.filterwarnings("ignore")

In [17]:

df = pd.read_csv("./datasets/mlr_spearman.csv")

In [18]:

df.head()

,target,age_at_diagnosis,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2ML1,A2MP1,A4GNT,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,inss_stage_2,inss_stage_3,inss_stage_4,inss_stage_4S
0,True,987,0.834381,1.910053,1.453191,0.005102,0.580151,0.027558,0.170671,0.057101,...,3.466692,0.862512,3.561729,5.039327,3.245179,3.296496,0,0,1,0
1,False,1808,0.743094,0.941996,0.640614,0.005902,0.738233,0.000000,0.000000,0.066125,...,3.201112,0.051655,3.595790,5.144157,2.843600,3.052771,0,0,1,0
2,False,625,0.909414,1.950857,1.156765,0.005192,0.927667,0.028036,0.045284,0.000000,...,3.056684,0.022912,2.236695,4.982975,2.880531,2.431521,0,0,1,0
3,False,335,0.795775,1.989477,1.525277,0.000000,0.936497,0.000000,0.044204,0.028540,...,3.081720,0.108028,3.312516,6.142091,2.864198,3.119886,1,0,0,0
4,True,536,0.905540,1.942946,1.365043,0.025347,0.924853,0.000000,0.000000,0.032056,...,2.823576,0.049621,2.235632,5.236531,2.761698,2.226502,0,0,1,0


In [19]:
X = df.drop('target',axis=1)
y = df['target']
y


0       True
1      False
2      False
3      False
4       True
       ...  
159    False
160    False
161    False
162     True
163    False
Name: target, Length: 164, dtype: bool

Run query to filter dataset - procude a binary mask to filter out genes accoridng to filter e.g filter = df_genes['ran_imp'] > 0.0

In [20]:
file = 'ADA_RANKED_1113_101.csv'
# file = 'RAN_RANKED_102.csv'
filepath = f'./csvs/gene_importances/{file}'
df_genes = pd.read_csv(filepath)
filter = df_genes['ada_imp'] > 0.0
# filter = df_genes['ran_imp'] > 0.0
df_genes = df_genes[filter]
df_genes

,gene,ada_imp,direction,ada_rank
0,TPRN,0.08,positive,1
1,CIC,0.06,positive,2
2,DTNA,0.06,positive,3
3,LOC100130872,0.06,positive,4
4,ANGPTL3,0.06,positive,5
5,RMND5A,0.06,positive,6
6,EIF5B,0.04,positive,7
7,C3orf67,0.04,positive,8
8,FLJ30679,0.04,positive,9
9,CFLAR-AS1,0.04,positive,10


In [21]:
gene_list = df_genes['gene'].values.tolist()
print(len(gene_list))
gene_list

27


['TPRN',
 'CIC',
 'DTNA',
 'LOC100130872',
 'ANGPTL3',
 'RMND5A',
 'EIF5B',
 'C3orf67',
 'FLJ30679',
 'CFLAR-AS1',
 'IGHMBP2',
 'PIGN',
 'TAB1',
 'MREG',
 'FLJ31813',
 'RTN4R',
 'ADCY10',
 'C9orf78',
 'KLHL22',
 'GREM2',
 'GRIK3',
 'FTH1P3',
 'ALAS1',
 'DLK1',
 'TCF21',
 'CRYAB',
 'SRR']

In [22]:
# add baCK target column to list
gene_list.append('target')
gene_list


['TPRN',
 'CIC',
 'DTNA',
 'LOC100130872',
 'ANGPTL3',
 'RMND5A',
 'EIF5B',
 'C3orf67',
 'FLJ30679',
 'CFLAR-AS1',
 'IGHMBP2',
 'PIGN',
 'TAB1',
 'MREG',
 'FLJ31813',
 'RTN4R',
 'ADCY10',
 'C9orf78',
 'KLHL22',
 'GREM2',
 'GRIK3',
 'FTH1P3',
 'ALAS1',
 'DLK1',
 'TCF21',
 'CRYAB',
 'SRR',
 'target']

In [24]:
df_reduced = df.filter(gene_list)



In [25]:
df

,target,age_at_diagnosis,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2ML1,A2MP1,A4GNT,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,inss_stage_2,inss_stage_3,inss_stage_4,inss_stage_4S
0,True,987,0.834381,1.910053,1.453191,0.005102,0.580151,0.027558,0.170671,0.057101,...,3.466692,0.862512,3.561729,5.039327,3.245179,3.296496,0,0,1,0
1,False,1808,0.743094,0.941996,0.640614,0.005902,0.738233,0.000000,0.000000,0.066125,...,3.201112,0.051655,3.595790,5.144157,2.843600,3.052771,0,0,1,0
2,False,625,0.909414,1.950857,1.156765,0.005192,0.927667,0.028036,0.045284,0.000000,...,3.056684,0.022912,2.236695,4.982975,2.880531,2.431521,0,0,1,0
3,False,335,0.795775,1.989477,1.525277,0.000000,0.936497,0.000000,0.044204,0.028540,...,3.081720,0.108028,3.312516,6.142091,2.864198,3.119886,1,0,0,0
4,True,536,0.905540,1.942946,1.365043,0.025347,0.924853,0.000000,0.000000,0.032056,...,2.823576,0.049621,2.235632,5.236531,2.761698,2.226502,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,False,162,0.845622,1.942807,1.089871,0.006957,1.665634,0.012612,0.061050,0.221767,...,3.580700,0.030691,2.957640,4.675505,3.217231,3.603715,0,0,1,0
160,False,865,0.997977,2.336929,1.182908,0.024298,0.673627,0.000000,0.053645,0.034626,...,3.523813,0.026980,2.827654,5.103993,3.519328,2.563558,0,0,1,0
161,False,1759,1.003559,2.836360,1.367371,0.007295,1.401265,0.013213,0.182636,0.000000,...,3.123189,0.094324,2.569946,4.754433,2.834434,2.415940,0,0,1,0
162,True,2326,0.842437,1.205317,0.643751,0.000000,0.837443,0.000000,0.000000,0.036918,...,3.018102,0.780477,3.569224,4.938587,2.949716,2.957478,0,0,1,0


In [26]:
df_reduced

,TPRN,CIC,DTNA,LOC100130872,ANGPTL3,RMND5A,EIF5B,C3orf67,FLJ30679,CFLAR-AS1,...,KLHL22,GREM2,GRIK3,FTH1P3,ALAS1,DLK1,TCF21,CRYAB,SRR,target
0,2.352315,5.017036,3.441078,0.182999,0.023631,3.634547,3.819321,1.487301,0.075259,0.000000,...,3.995702,0.730560,0.308878,1.816444,3.447385,0.472023,0.498505,2.880517,2.807005,True
1,3.100552,5.477470,3.872691,0.139642,0.027392,3.536811,3.745280,1.124143,0.087020,0.120218,...,4.115491,0.344939,0.831351,1.665479,3.800983,1.248638,0.351506,2.469451,3.195365,False
2,2.546811,4.910598,3.716804,0.104565,0.024023,3.714586,4.139911,0.720889,0.125302,0.000000,...,4.024346,0.932035,0.564331,1.591679,3.884686,1.677242,0.243135,3.184685,2.261747,False
3,2.568081,4.988067,3.657686,0.065700,0.134915,3.775883,3.837247,0.387879,0.234647,0.102811,...,4.248239,0.317690,0.663855,1.378545,4.212546,4.756746,1.476848,3.529821,2.478467,False
4,2.554471,4.664818,2.729762,0.219258,0.026270,3.296546,4.166145,0.461690,0.056205,0.058720,...,3.928304,0.192882,0.167089,1.783574,4.041287,5.872479,0.000000,6.228613,2.503458,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,2.307644,5.116606,4.108474,0.039370,0.063860,3.969906,3.504214,0.722269,0.034877,0.000000,...,4.169476,0.247059,2.924818,1.020548,3.018453,7.993584,0.636233,3.501414,2.406504,False
160,2.636116,4.416705,3.868262,0.057144,0.028364,3.824208,2.995224,0.424533,0.030643,0.063574,...,3.772477,0.145492,2.090308,1.000613,3.619941,4.271508,0.287322,2.692066,2.005950,False
161,2.615367,3.409850,2.917128,0.094057,0.000000,3.832880,3.132666,0.992917,0.036413,0.146425,...,4.146223,0.186625,3.626591,1.166921,4.633565,10.848584,0.522945,3.221238,2.451878,False
162,2.761338,4.606573,5.043178,0.155375,0.000000,3.334088,3.921541,1.940836,0.000000,0.000000,...,3.855591,0.155274,0.045522,1.736964,4.623615,0.464654,0.178927,1.545113,3.124231,True


In [27]:
output_file = f'RED_{file}'
output = f'./datasets/reduced_features_mlr_datasets/{output_file}' 
output

'./datasets/reduced_features_mlr_datasets/RED_ADA_RANKED_1113_101.csv'

In [ ]:
df.to_csv(output, index=False)
